Есть Pandas DataFrame со столбцами:
 - customer_id, 
 - product_id
 - timestamp
 
Датафрейм содержит данные по просмотрам товаров на сайте. Есть проблема – просмотры одного customer_id не разбиты на сессии (появления на сайте). Мы хотим разместить сессии так, чтобы сессией считались все смежные просмотры, между которыми не более 3 минут.

Написать методом который создаст в Pandas DataFrame столбец session_id и проставит в нем уникальный int id для каждой сессии.

У каждого пользователя может быть по несколько сессий. Исходный DataFrame может быть большим – до 100 млн строк.

## Сгенерируем данные (вручную)

Для проверки корректности работы методов

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm


df = pd.DataFrame(data=[[1, 1, '2022/11/06 17:45:30.005'],
                        [1, 3, '2022/11/06 17:46:48.223'],
                        [1, 4, '2022/11/06 17:49:48.005'],
                        [2, 43, '2022/09/14 21:03:25.543'],
                        [1, 1, '2022/11/06 17:55:12.022'],
                        [4, 1, '2022/11/06 17:56:10.022'],
                        [1, 13, '2022/11/06 17:56:12.022'],
                        [1, 24, '2022/11/06 17:59:25.022']],
                  columns=['customer_id', 'product_id', 'timestamp'])
df.customer_id = df.customer_id.astype(int)
df.product_id = df.product_id.astype(int)
df.timestamp = pd.to_datetime(df.timestamp)
df

,customer_id,product_id,timestamp
0,1,1,2022-11-06 17:45:30.005
1,1,3,2022-11-06 17:46:48.223
2,1,4,2022-11-06 17:49:48.005
3,2,43,2022-09-14 21:03:25.543
4,1,1,2022-11-06 17:55:12.022
5,4,1,2022-11-06 17:56:10.022
6,1,13,2022-11-06 17:56:12.022
7,1,24,2022-11-06 17:59:25.022


Заметим, что ключевым `customer_id` будет 1. На нем визуально проверим все "узкие моменты" на специально сгенерированных данных.

## 1ое решение

Проходимся одинарным циклом по всему фрейму и используем словари (т.к. сложность O(1)) для сохранения информации.

Опять же, можно было использовать `.itertuples()` вместо `.iterrows()` для немного бОльшей эффективности.

In [2]:
%%time

class Frame:
    def __init__(self, df):
        df_copy = df.copy()
        self.df = df_copy


class Frame1(Frame):
    
    '''Первый вариант решения'''
    
    def get_session_id(self):
        d = {}
        d_c = {}
        for i, val in self.df.iterrows():
            if val.customer_id not in d:
                d[val.customer_id] = val.timestamp
                d_c[val.customer_id] = 1
                self.df.loc[i, 'session_id'] = 1
            else:
                if pd.Timedelta(val.timestamp - d[val.customer_id]).seconds / 60.0 > 3.0:
                    d_c[val.customer_id] += 1
                d[val.customer_id] = val.timestamp
                self.df.loc[i, 'session_id'] = d_c[val.customer_id]
        self.df.session_id = self.df.session_id.astype(int)
        return self.df


df_1 = Frame1(df)
df_1.get_session_id()

CPU times: user 5.3 ms, sys: 313 µs, total: 5.62 ms
Wall time: 9.86 ms


,customer_id,product_id,timestamp,session_id
0,1,1,2022-11-06 17:45:30.005,1
1,1,3,2022-11-06 17:46:48.223,1
2,1,4,2022-11-06 17:49:48.005,1
3,2,43,2022-09-14 21:03:25.543,1
4,1,1,2022-11-06 17:55:12.022,2
5,4,1,2022-11-06 17:56:10.022,1
6,1,13,2022-11-06 17:56:12.022,2
7,1,24,2022-11-06 17:59:25.022,3


## 2ое решение

Используем методы `pandas`

In [3]:
%%time

class Frame2(Frame):
    
    '''Второй вариант решения'''
    
    def get_session_id(self):
        self.df['Diff'] = (self.df.groupby(['customer_id'])['timestamp'].diff()
                           .dt.seconds / 60 ).astype(float).gt(3)
        self.df['session_id'] = 1 + self.df.groupby('customer_id')['Diff'].cumsum()
        self.df.drop('Diff', axis=1, inplace=True)
        return self.df


df_2 = Frame2(df)
df_2.get_session_id()

CPU times: user 13.3 ms, sys: 0 ns, total: 13.3 ms
Wall time: 11.9 ms


,customer_id,product_id,timestamp,session_id
0,1,1,2022-11-06 17:45:30.005,1
1,1,3,2022-11-06 17:46:48.223,1
2,1,4,2022-11-06 17:49:48.005,1
3,2,43,2022-09-14 21:03:25.543,1
4,1,1,2022-11-06 17:55:12.022,2
5,4,1,2022-11-06 17:56:10.022,1
6,1,13,2022-11-06 17:56:12.022,2
7,1,24,2022-11-06 17:59:25.022,3


Как итог, оба решения отработали корректно. Получили предполагаемый результат.